In [2]:
import os, sys

os.chdir("../../..")
print(os.path.abspath("./"))

/data2/huping/home/projects/src/github.com/any35/MOS


In [2]:
# convert mmae-dataset to 1.5mm resolution
import torch, glob, os
from torchvision.transforms import functional as F, InterpolationMode

os.makedirs(".cache/dataset/mmae-dataset/split-1.5mm", exist_ok=True)

total_image_count = 0
total_pair_count = 0
file_list = glob.glob(".cache/dataset/mmae-dataset/split/*.pt")
file_list.sort()

for file in file_list:
    data = torch.load(file)
    image, pair = data["image"], data["pair"]

    image = image.to(torch.uint8)
    image = F.resize(
        image,
        (214, 214),
        interpolation=InterpolationMode.BILINEAR,
        antialias=True,
    )
    total_image_count += image.shape[0]
    total_pair_count += pair.shape[0]
    print(f"processed {file}...\t {image.shape[0]} images, { pair.shape[0]} pairs")

    data["image"] = image
    file_name = os.path.basename(file)

    torch.save(
        data,
        f".cache/dataset/mmae-dataset/split-1.5mm/{file_name}",
    )

print(f"{total_image_count} images & {total_pair_count} pairs saved!")

processed .cache/dataset/mmae-dataset/split/ACDC.pt...	 46387 images, 46387 pairs
processed .cache/dataset/mmae-dataset/split/CMRI-private.pt...	 48195 images, 49662 pairs
processed .cache/dataset/mmae-dataset/split/CMRI.pt...	 30951 images, 30951 pairs
processed .cache/dataset/mmae-dataset/split/CT_MR_2D_Dataset_DA.pt...	 9403 images, 9403 pairs
processed .cache/dataset/mmae-dataset/split/CoA_MRIData.pt...	 2641 images, 2641 pairs
processed .cache/dataset/mmae-dataset/split/DOI_10.7910_DVN_CI3WB6.pt...	 42225 images, 42225 pairs
processed .cache/dataset/mmae-dataset/split/DOI_10.7910_DVN_JMZHVI.pt...	 16000 images, 16000 pairs
processed .cache/dataset/mmae-dataset/split/DOI_10.7910_DVN_N1R1Q4.pt...	 1050 images, 1050 pairs
processed .cache/dataset/mmae-dataset/split/EMIDEC.pt...	 2482 images, 2482 pairs
processed .cache/dataset/mmae-dataset/split/HVSMR2016.pt...	 6382 images, 6382 pairs
processed .cache/dataset/mmae-dataset/split/LeftAtrialSegmentationChallenge2013.pt...	 6031 images,

In [3]:
# filter segment only & pair
import torch, glob
from torchvision.transforms import functional as F, InterpolationMode

os.makedirs(".cache/dataset/mmae-dataset/split-1.5mm-segment-only", exist_ok=True)

file_list = glob.glob(".cache/dataset/mmae-dataset/split-1.5mm/*.pt")
file_list.sort()

total_image_count = 0
total_pair_count = 0


def is_label(modality: int):
    return modality >= 11 and modality <= 20


def is_any_label(a: int, b: int, c: int, d: int, e: int):
    return is_label(a) or is_label(b) or is_label(c) or is_label(d) or is_label(e)


for file in file_list:
    file_name = file.split("/")[-1]
    data = torch.load(file)

    new_image_index = []
    new_image_mapping = dict()

    new_pair_list = []
    image, pair = data["image"], data["pair"]
    pair_list = pair.tolist()
    for pair in pair.tolist():
        src_modality, src_index, target_modality, target_index = pair
        if not is_label(target_modality):
            continue

        if src_index not in new_image_mapping:
            new_image_mapping[src_index] = len(new_image_index)
            new_image_index.append(src_index)
        if target_index not in new_image_mapping:
            new_image_mapping[target_index] = len(new_image_index)
            new_image_index.append(target_index)

        src_index = new_image_mapping[src_index]
        target_index = new_image_mapping[target_index]
        new_pair_list.append([src_modality, src_index, target_modality, target_index])
    data["pair"] = torch.tensor(new_pair_list)

    for merged_pair_key in ["merged_pair", "all_merged_pair"]:
        if merged_pair_key not in data:
            continue
        new_merged_pair_list = []
        for merged_pair in data[merged_pair_key].tolist():
            src_modality, src_index, attr1, attr2, attr3, attr4, attr5, idx1, idx2, idx3, idx4, idx5 = merged_pair
            if not is_any_label(attr1, attr2, attr3, attr4, attr5):
                continue
            if src_index not in new_image_mapping:
                new_image_mapping[src_index] = len(new_image_index)
                new_image_index.append(src_index)
            src_index = new_image_mapping[src_index]
            if is_label(attr1):
                if idx1 not in new_image_mapping:
                    new_image_mapping[idx1] = len(new_image_index)
                    new_image_index.append(idx1)
                idx1 = new_image_mapping[idx1]
            if is_label(attr2):
                if idx2 not in new_image_mapping:
                    new_image_mapping[idx2] = len(new_image_index)
                    new_image_index.append(idx2)
                idx2 = new_image_mapping[idx2]
            if is_label(attr3):
                if idx3 not in new_image_mapping:
                    new_image_mapping[idx3] = len(new_image_index)
                    new_image_index.append(idx3)
                idx3 = new_image_mapping[idx3]
            if is_label(attr4):
                if idx4 not in new_image_mapping:
                    new_image_mapping[idx4] = len(new_image_index)
                    new_image_index.append(idx4)
                idx4 = new_image_mapping[idx4]
            if is_label(attr5):
                if idx5 not in new_image_mapping:
                    new_image_mapping[idx5] = len(new_image_index)
                    new_image_index.append(idx5)
                idx5 = new_image_mapping[idx5]
            new_merged_pair_list.append(
                [
                    src_modality,
                    src_index,
                    attr1,
                    attr2,
                    attr3,
                    attr4,
                    attr5,
                    idx1,
                    idx2,
                    idx3,
                    idx4,
                    idx5,
                ]
            )

        data[merged_pair_key] = torch.tensor(new_merged_pair_list)

    data["image_meta"] = data["image_meta"][new_image_index]
    data["image"] = image[new_image_index]
    if len(new_image_index) > 0:
        torch.save(
            data,
            f".cache/dataset/mmae-dataset/split-1.5mm-segment-only/{file_name}",
        )
    print(f"processed {file_name}...\t {len(new_image_index)} images, {len(new_pair_list)} pairs")
    total_image_count += len(new_image_index)
    total_pair_count += len(new_pair_list)

print(f"total {total_image_count} train images, {total_pair_count} pairs!")

processed ACDC.pt...	 10883 images, 8041 pairs
processed CMRI-private.pt...	 0 images, 0 pairs
processed CMRI.pt...	 3002 images, 1501 pairs
processed CT_MR_2D_Dataset_DA.pt...	 9403 images, 7835 pairs
processed CoA_MRIData.pt...	 0 images, 0 pairs
processed DOI_10.7910_DVN_CI3WB6.pt...	 0 images, 0 pairs
processed DOI_10.7910_DVN_JMZHVI.pt...	 0 images, 0 pairs
processed DOI_10.7910_DVN_N1R1Q4.pt...	 0 images, 0 pairs
processed EMIDEC.pt...	 2124 images, 1416 pairs
processed HVSMR2016.pt...	 4145 images, 2782 pairs
processed LeftAtrialSegmentationChallenge2013.pt...	 4426 images, 2463 pairs
processed LeftAtrialSegmentationKaggle.pt...	 2702 images, 1351 pairs
processed MMWHS2017.pt...	 33300 images, 26628 pairs
processed MyoPS2020.pt...	 597 images, 873 pairs
processed RegionalMulti-viewLearningForCardiacMotionAnalysis.pt...	 0 images, 0 pairs
processed SunnybrookCardiacData.pt...	 0 images, 0 pairs
processed VarDA.pt...	 3785 images, 2779 pairs
processed mnms.pt...	 21649 images, 159

In [ ]:
# spit segment only CMRT dataset into 10 folds
import torch, glob, os

data = torch.load(".cache/dataset/mmae-dataset/split-1.5mm-segment-only/CMRI.pt")


def is_label(modality: int):
    return modality >= 11 and modality <= 20


def is_any_label(a: int, b: int, c: int, d: int, e: int):
    return is_label(a) or is_label(b) or is_label(c) or is_label(d) or is_label(e)


image, pair, image_meta, merged_pair, all_merged_pair = (
    data["image"],
    data["pair"],
    data["image_meta"],
    data["merged_pair"],
    data["all_merged_pair"],
)

new_image_index = [[] for _ in range(10)]
new_image_mapping = [dict() for _ in range(10)]
new_pair_list = [[] for _ in range(10)]
new_merge_pair_list = {
    "merged_pair": [[] for _ in range(10)],
    "all_merged_pair": [[] for _ in range(10)],
}

for pair in pair.tolist():
    src_modality, src_index, target_modality, target_index = pair
    uid = image_meta[src_index][0]
    partition = uid % 10
    if src_index not in new_image_mapping[partition]:
        new_image_mapping[partition][src_index] = len(new_image_index[partition])
        new_image_index[partition].append(src_index)
    if target_index not in new_image_mapping[partition]:
        new_image_mapping[partition][target_index] = len(new_image_index[partition])
        new_image_index[partition].append(target_index)

    src_index = new_image_mapping[partition][src_index]
    target_index = new_image_mapping[partition][target_index]
    new_pair_list[partition].append([src_modality, src_index, target_modality, target_index])

for merged_pair_key in ["merged_pair", "all_merged_pair"]:
    for merged_pair in data[merged_pair_key].tolist():
        src_modality, src_index, attr1, attr2, attr3, attr4, attr5, idx1, idx2, idx3, idx4, idx5 = merged_pair
        uid = image_meta[src_index][0]
        partition = uid % 10

        if not is_any_label(attr1, attr2, attr3, attr4, attr5):
            continue
        if src_index not in new_image_mapping[partition]:
            new_image_mapping[partition][src_index] = len(new_image_index[partition])
            new_image_index[partition].append(src_index)
        src_index = new_image_mapping[partition][src_index]
        if is_label(attr1):
            if idx1 not in new_image_mapping[partition]:
                new_image_mapping[partition][idx1] = len(new_image_index[partition])
                new_image_index[partition].append(idx1)
            idx1 = new_image_mapping[partition][idx1]
        if is_label(attr2):
            if idx2 not in new_image_mapping[partition]:
                new_image_mapping[partition][idx2] = len(new_image_index[partition])
                new_image_index[partition].append(idx2)
            idx2 = new_image_mapping[partition][idx2]
        if is_label(attr3):
            if idx3 not in new_image_mapping[partition]:
                new_image_mapping[partition][idx3] = len(new_image_index[partition])
                new_image_index[partition].append(idx3)
            idx3 = new_image_mapping[partition][idx3]
        if is_label(attr4):
            if idx4 not in new_image_mapping[partition]:
                new_image_mapping[partition][idx4] = len(new_image_index[partition])
                new_image_index[partition].append(idx4)
            idx4 = new_image_mapping[partition][idx4]
        if is_label(attr5):
            if idx5 not in new_image_mapping[partition]:
                new_image_mapping[partition][idx5] = len(new_image_index[partition])
                new_image_index[partition].append(idx5)
            idx5 = new_image_mapping[partition][idx5]
        new_merge_pair_list[merged_pair_key][partition].append(
            [
                src_modality,
                src_index,
                attr1,
                attr2,
                attr3,
                attr4,
                attr5,
                idx1,
                idx2,
                idx3,
                idx4,
                idx5,
            ]
        )

for partition in range(10):
    p_image = image[new_image_index[partition]]
    p_image_meta = image_meta[new_image_index[partition]]
    p_pair = torch.tensor(new_pair_list[partition])
    p_merged_pair = torch.tensor(new_merge_pair_list["merged_pair"][partition])
    p_all_merged_pair = torch.tensor(new_merge_pair_list["all_merged_pair"][partition])
    torch.save(
        {
            "image": p_image,
            "pair": p_pair,
            "image_meta": p_image_meta,
            "merged_pair": p_merged_pair,
            "all_merged_pair": p_all_merged_pair,
        },
        f".cache/dataset/mmae-dataset/split-1.5mm-segment-only/CMRI-{partition}.pt",
    )
    print(f"partition {partition}, {len(p_image)} images, {len(p_pair)} pairs")
del data, image, pair, image_meta, merged_pair, all_merged_pair
del new_image_index, new_image_mapping, new_pair_list, new_merge_pair_list

partition 0, 298 images, 149 pairs
partition 1, 272 images, 136 pairs
partition 2, 308 images, 154 pairs
partition 3, 300 images, 150 pairs
partition 4, 304 images, 152 pairs
partition 5, 324 images, 162 pairs
partition 6, 290 images, 145 pairs
partition 7, 324 images, 162 pairs
partition 8, 292 images, 146 pairs
partition 9, 290 images, 145 pairs


In [2]:
# filter image only & pair
import torch, glob
from torchvision.transforms import functional as F, InterpolationMode

os.makedirs(".cache/dataset/mmae-dataset/split-1.5mm-image-only", exist_ok=True)

file_list = glob.glob(".cache/dataset/mmae-dataset/split-1.5mm/*.pt")
file_list.sort()

total_image_count = 0
total_pair_count = 0


def is_label(modality: int):
    return modality >= 11 and modality <= 20


for file in file_list:
    file_name = file.split("/")[-1]
    data = torch.load(file)

    new_image_index = []
    new_image_mapping = dict()

    new_pair_list = []
    image, pair = data["image"], data["pair"]
    pair_list = pair.tolist()
    for pair in pair.tolist():
        src_modality, src_index, target_modality, target_index = pair
        if is_label(target_modality):
            continue

        if src_index not in new_image_mapping:
            new_image_mapping[src_index] = len(new_image_index)
            new_image_index.append(src_index)
        if target_index not in new_image_mapping:
            new_image_mapping[target_index] = len(new_image_index)
            new_image_index.append(target_index)

        src_index = new_image_mapping[src_index]
        target_index = new_image_mapping[target_index]
        new_pair_list.append([src_modality, src_index, target_modality, target_index])
    data["pair"] = torch.tensor(new_pair_list)

    data["image_meta"] = data["image_meta"][new_image_index]
    data["image"] = image[new_image_index]
    if len(new_image_index) > 0:
        torch.save(
            data,
            f".cache/dataset/mmae-dataset/split-1.5mm-image-only/{file_name}",
        )
    print(f"processed {file_name}...\t {len(new_image_index)} images, {len(new_pair_list)} pairs")
    total_image_count += len(new_image_index)
    total_pair_count += len(new_pair_list)

print(f"total {total_image_count} train images, {total_pair_count} pairs!")

processed ACDC.pt...	 38346 images, 38346 pairs
processed CMRI-private.pt...	 48195 images, 49662 pairs
processed CMRI.pt...	 29450 images, 29450 pairs
processed CT_MR_2D_Dataset_DA.pt...	 1568 images, 1568 pairs
processed CoA_MRIData.pt...	 2641 images, 2641 pairs
processed DOI_10.7910_DVN_CI3WB6.pt...	 42225 images, 42225 pairs
processed DOI_10.7910_DVN_JMZHVI.pt...	 16000 images, 16000 pairs
processed DOI_10.7910_DVN_N1R1Q4.pt...	 1050 images, 1050 pairs
processed EMIDEC.pt...	 1066 images, 1066 pairs
processed HVSMR2016.pt...	 3600 images, 3600 pairs
processed LeftAtrialSegmentationChallenge2013.pt...	 3568 images, 3568 pairs
processed LeftAtrialSegmentationKaggle.pt...	 2271 images, 2271 pairs
processed MMWHS2017.pt...	 23980 images, 23980 pairs
processed MyoPS2020.pt...	 522 images, 1566 pairs
processed RegionalMulti-viewLearningForCardiacMotionAnalysis.pt...	 4101 images, 4101 pairs
processed SunnybrookCardiacData.pt...	 51545 images, 51545 pairs
processed VarDA.pt...	 1077 imag

In [4]:
# spit image only CMRT dataset into 10 folds
import torch, glob, os

data = torch.load(".cache/dataset/mmae-dataset/split-1.5mm-image-only/CMRI.pt")


image, pair, image_meta = (
    data["image"],
    data["pair"],
    data["image_meta"],
)

new_image_index = [[] for _ in range(10)]
new_image_mapping = [dict() for _ in range(10)]
new_pair_list = [[] for _ in range(10)]

for pair in pair.tolist():
    src_modality, src_index, target_modality, target_index = pair
    uid = image_meta[src_index][0]
    partition = uid % 10
    if src_index not in new_image_mapping[partition]:
        new_image_mapping[partition][src_index] = len(new_image_index[partition])
        new_image_index[partition].append(src_index)
    if target_index not in new_image_mapping[partition]:
        new_image_mapping[partition][target_index] = len(new_image_index[partition])
        new_image_index[partition].append(target_index)

    src_index = new_image_mapping[partition][src_index]
    target_index = new_image_mapping[partition][target_index]
    new_pair_list[partition].append([src_modality, src_index, target_modality, target_index])


for partition in range(10):
    p_image = image[new_image_index[partition]]
    p_image_meta = image_meta[new_image_index[partition]]
    p_pair = torch.tensor(new_pair_list[partition])
    torch.save(
        {
            "image": p_image,
            "pair": p_pair,
            "image_meta": p_image_meta,
        },
        f".cache/dataset/mmae-dataset/split-1.5mm-image-only/CMRI-{partition}.pt",
    )
    print(f"partition {partition}, {len(p_image)} images, {len(p_pair)} pairs")
del data, image, pair, image_meta
del new_image_index, new_image_mapping, new_pair_list

partition 0, 3090 images, 3090 pairs
partition 1, 2340 images, 2340 pairs
partition 2, 3000 images, 3000 pairs
partition 3, 3210 images, 3210 pairs
partition 4, 2970 images, 2970 pairs
partition 5, 2850 images, 2850 pairs
partition 6, 2970 images, 2970 pairs
partition 7, 3120 images, 3120 pairs
partition 8, 3070 images, 3070 pairs
partition 9, 2830 images, 2830 pairs


In [5]:
# make mmae dataset

# all image + train pair + valid pair + train_cmri_pair/{i} + valid_cmri_pair/{i}

import torch, glob, os


exclude_files = set(["CMRI.pt", "token.pt"])
for i in range(10):
    exclude_files.add(f"CMRI-{i}.pt")

valid_files = set(["VarDA.pt"])

file_list = glob.glob(".cache/dataset/mmae-dataset/split-1.5mm/*.pt")
file_list.sort()

image_list = [
    torch.zeros(1, 214, 214, dtype=torch.uint8),  # zero padding
]

# 各target_modality数量统计
#  (0,2676),
#  (1,215162),
#  (2,1260),
#  (3,96329),
#  (4,1494),
#  (5,11571),
#  (6,16404),
#  (7,16000),
#  (11,19384),
#  (12,13804),
#  (13,21339),
#  (14,9366),
#  (15,5343),
#  (16,4207),
#  (17,4644),
#  (18,634),

# 用于平衡各个target_modality的数量
repeat_count = dict(
    [
        (0, 1),
        (1, 1),  # T1
        (2, 100),  # T2
        (3, 10),  # CINE
        (4, 100),  # LGE
        (5, 10),  # SSFP
        (6, 1),  # CT
        (7, 1),  # Flair
        (9, 1),  # Ultrasound
        (11, 10),  # LV
        (12, 10),  # RV
        (13, 10),  # MYO
        (14, 10),  # LA
        (15, 10),  # RA
        (16, 10),  # AO
        (17, 10),  # PA
        (18, 10),  # VA
    ]
)

train_pair_list = []
valid_pair_list = []
all_cmri_pair_list = []
train_cmri_list = [[] for _ in range(10)]
valid_cmri_list = [[] for _ in range(10)]

index = 1
for file in file_list:
    file_name = file.split("/")[-1]
    if file_name in exclude_files:
        continue

    is_valid = file_name in valid_files

    data = torch.load(file)
    image, pair = data["image"], data["pair"]
    pair[:, 1] += index
    pair[:, 3] += index

    # 用于平衡各个target_modality的数量
    pair_index = []
    for i, modality in enumerate(pair[:, 2].tolist()):
        pair_index.extend([i] * repeat_count[modality])
    pair = pair[pair_index]

    image_list.append(image)
    index += image.shape[0]
    if is_valid:
        print(f"valid file: {file}")
        valid_pair_list.append(pair)
    else:
        print(f"train file: {file}")
        train_pair_list.append(pair)


# 划分数据集, 做交叉测试
data = torch.load(".cache/dataset/mmae-dataset/split-1.5mm/CMRI.pt")
image, image_meta, pair = data["image"], data["image_meta"], data["pair"]
image_meta = image_meta.tolist()
for pair in pair.tolist():
    src_modality, src_index, target_modality, target_index = pair
    uid = image_meta[src_index][0]
    partition = uid % 10
    src_index += index
    target_index += index
    if target_modality != 20:  # 因为训练集没有心包脂肪, 所以验证集也不需要验证脂肪
        all_cmri_pair_list.append([src_modality, src_index, target_modality, target_index])
    for i in range(10):
        if i == partition:
            valid_cmri_list[i].append([src_modality, src_index, target_modality, target_index])
        else:
            train_cmri_list[i].append([src_modality, src_index, target_modality, target_index])
image_list.append(image)
index += image.shape[0]

image_list = torch.cat(image_list, dim=0).to(torch.uint8)
train_pair_list = torch.cat(train_pair_list, 0).int()
valid_pair_list = torch.cat(valid_pair_list, 0).int()
all_cmri_pair_list = torch.tensor(all_cmri_pair_list).int()
train_cmri_list = [torch.tensor(train_cmri_list[i]).int() for i in range(10)]
valid_cmri_list = [torch.tensor(valid_cmri_list[i]).int() for i in range(10)]

assert len(image_list.shape) == 3
assert len(train_pair_list.shape) == 2 and train_pair_list.shape[1] == 4
assert len(valid_pair_list.shape) == 2 and valid_pair_list.shape[1] == 4
assert len(all_cmri_pair_list.shape) == 2 and all_cmri_pair_list.shape[1] == 4
assert len(train_cmri_list[0].shape) == 2 and train_cmri_list[0].shape[1] == 4

data = {
    "image": image_list,  # (bs, h, w)
    "train_pair": train_pair_list,  # (bs, [src_modality, src_index, target_modality, target_index])
    "valid_pair": valid_pair_list,  # (bs, [src_modality, src_index, target_modality, target_index])
    "all_cmri_pair": all_cmri_pair_list,  # (bs, [src_modality, src_index, target_modality, target_index])
}
for i in range(10):
    # (bs, [src_modality, src_index, target_modality, target_index])
    # finetune的时候使用, 10 folds
    data[f"train_cmri_pair/{i}"] = train_cmri_list[i]
    data[f"valid_cmri_pair/{i}"] = valid_cmri_list[i]

torch.save(
    data,
    f".cache/dataset/mmae-dataset/dataset.pt",
)
print(
    f"processed dataset.pt completed! {image_list.shape[0]} images, {train_pair_list.shape[0]} train pairs, {valid_pair_list.shape[0]} valid pairs, {all_cmri_pair_list.shape[0]} all cmri pairs"
)

print("统计各个target_modality的数量")
modalities = train_pair_list[:, 2].unique()
for mod in modalities:
    print(mod, "=", (train_pair_list[:, 2] == mod).sum().item())

train file: .cache/dataset/mmae-dataset/split-1.5mm/ACDC.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/CMRI-private.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/CT_MR_2D_Dataset_DA.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/CoA_MRIData.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_CI3WB6.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_JMZHVI.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_N1R1Q4.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/EMIDEC.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/HVSMR2016.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/LeftAtrialSegmentationChallenge2013.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/LeftAtrialSegmentationKaggle.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/MMWHS2017.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/MyoPS2020.pt
train file: .cache/dataset/mmae-dataset/split-1.5mm/RegionalMult

In [2]:
# show dataset
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact

file = ".cache/dataset/mmae-dataset/dataset.pt"
data = torch.load(file)
(
    all_image,  # (bs, h, w)
    train_pair,  # (bs, [src_modality, src_index, target_modality, target_index])
    valid_pair,
    all_cmri_pair,  # (bs, [src_modality, src_index, target_modality, target_index])
    train_cmri_pair,
    valid_cmri_pair,
) = (
    data["image"],
    data["train_pair"],
    data["valid_pair"],
    data["all_cmri_pair"],
    data["train_cmri_pair/0"],
    data["valid_cmri_pair/0"],
)


def draw_image_simple(pair, title=""):
    pair_count = pair.shape[0] - 1

    @interact
    def _draw_image(pair_index=(0, pair_count)):
        src_modality, src_index, target_modality, target_index = pair[pair_index].tolist()

        image = all_image[[src_index, target_index], ::]

        image = image.permute(1, 0, 2).reshape(214, -1)

        plt.title([title, src_modality, src_index, target_modality, target_index])
        plt.imshow(image, cmap="gray")

    return _draw_image

In [3]:
draw_image_simple(train_pair, "train_pair")
# draw_image_simple(valid_pair, "valid_pair")
# draw_image_simple(all_cmri_pair, "all_cmri_pair")
# draw_image_simple(train_cmri_pair, "train_cmri_pair")
# draw_image_simple(valid_cmri_pair, "valid_cmri_pair")

interactive(children=(IntSlider(value=219808, description='pair_index', max=439616), Output()), _dom_classes=(…

<function __main__.draw_image_simple.<locals>._draw_image(pair_index=(0, 439616))>